In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2

In [62]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from src.features import build_features
from src.visualization import visualize

sns.set_theme(palette="pastel")
plt.rcParams["figure.figsize"] = (12, 6)
pd.set_option("display.max_columns", None)

# Asumiendo que es una ocupacion del 80% de camas
INDICE_OCUPACION_CAMAS = 1.25

# Agrega las columnas de poblacion de interes
COLUMNAS_POBLACION_INTERES = {str(i): i for i in range(2017, 2036)}

# Proyección de casos INT al 2035

En este cuadernillo se quiere estimar la deminada que atenderá el INT para el 2035. Con esta demanada se calcularán las cantidades de camas necesarias para suplir tal demanda. Para determinar la demanda se utilizarán:

1. Cantidad de casos teóricos por problemas de salud relevantes seleccionados: Estos se calcularon según la incidencia de cada problema de salud y la proyección de la población al 2035.

2. Cantidad de casos atendidos actualmente por cada problema de salud: Estos se obtuvieron al analizar la base de egresos hospitalarios DEIS.

Con ambos datos, se obtendrá cuanto % de atención ha cubierto el INT de los casos teóricos.

In [3]:
# Obtiene los casos de area de influencia
casos_area_de_influencia = pd.read_excel(
    "../data/interim/casos_teoricos_diagnosticos.xlsx", sheet_name="casos_area_de_influencia_INT"
)

# Preprocesa el diagnostico
casos_area_de_influencia["Diagnostico"] = (
    casos_area_de_influencia["Diagnostico"].str.split(" - ").str[0]
)

# Preprocesa los diagnosticos agrupados
casos_area_de_influencia["Diagnosticos Contenidos"] = casos_area_de_influencia[
    "Diagnosticos Contenidos"
].str.split(", ")

# Renombra columnas de la poblacion
casos_area_de_influencia = casos_area_de_influencia.rename(columns=COLUMNAS_POBLACION_INTERES)

In [4]:
# Obtiene los diagnosticos relevantes a filtrar de los egresos hospitalarios
DIAGNOSTICOS_RELEVANTES = list(
    casos_area_de_influencia["Diagnostico"].str.split(" - ").str[0].unique()
)

# Obtiene los diagnosticos agrupados
DIAGNOSTICOS_AGRUPADOS = list(
    casos_area_de_influencia.explode("Diagnosticos Contenidos")["Diagnosticos Contenidos"]
    .dropna()
    .str.strip()
    .unique()
)

# Suma los diagnosticos agrupados a los diagnosticos relevantes
DIAGNOSTICOS_RELEVANTES += DIAGNOSTICOS_AGRUPADOS
DIAGNOSTICOS_RELEVANTES = sorted(DIAGNOSTICOS_RELEVANTES)

In [5]:
# Genera duplas de diagnosticos a reasignar
DIAGNOSTICOS_A_REASIGNAR = casos_area_de_influencia.query("`Diagnosticos Contenidos`.notna()")[
    ["Diagnostico", "Diagnosticos Contenidos"]
]

In [6]:
casos_area_de_influencia = casos_area_de_influencia.set_index("Diagnostico")

## Obtención de casos atendidos entre 2017 y 2020 para diagnósticos relevantes.

Estos se obtendrán del análisis de egresos hospitalarios a nivel país DEIS.

In [7]:
# Carga los egresos nacionales
egresos_nacionales = pd.read_csv(
    "../data/raw/4_resumen_egresos_nacionales/ranking_nacional_egresos.csv",
    sep=";",
    encoding="latin-1",
)

# Filtra solamente los datos del INT
egresos_torax = egresos_nacionales.query("ESTABLECIMIENTO_SALUD == 112103").copy()

En primer lugar, se filtrarán ambas bases de datos para solamente tener la información de los diagnósticos más relevantes para el INT. Luego de esto, se calculará la cantidad de egresos, dias de estada y pacientes para cada uno de los diagnósticos. Estos insumos serán utilizados para estimar la demanda.


In [8]:
# Filtra solamente los diagnosticos mas relevantes del Torax
egresos_mas_relevantes_nacionales = (
    egresos_nacionales[egresos_nacionales["DIAG1"].isin(DIAGNOSTICOS_RELEVANTES)]
    .query("ANO_EGRESO >= 2017 and ANO_EGRESO <= 2019")
    .copy()
)

## Reasginación de diagnósticos y agrupación

En este apartado se reasignarán diagnósticos de la base de datos DEIS a un grupo mayor. Por ejemplo,
los siguientes diagnósticos:

- C341 - Tumor maligno del lóbulo superior, bronquio o pulmón
- C342 - Tumor maligno del lóbulo medio, bronquio o pulmón
- C343 - Tumor maligno del lóbulo inferior, bronquio o pulmón
- C780 - Tumor maligno secundario del pulmón
- C782 - Tumor maligno secundario de la pleura
- D381 - Tumor de comportamiento incierto o desconocido de la tráquea, de los bronquios y del pulmón

Todos estos se reasignarán al diagnóstico C34XN (N, para que no se mezclen con los C34X asignados realmente así). Esto, con el fin de poder calcular las estadísticas agrupadas para todos estos diagnósticos (egresos por persona, días de estada).

In [9]:
for row in DIAGNOSTICOS_A_REASIGNAR.itertuples():
    diagnostico_nuevo = row[1]
    diagnosticos_antiguos = row[2]

    print(f"Cambiando {diagnosticos_antiguos} a {diagnostico_nuevo}")
    egresos_mas_relevantes_nacionales["DIAG1"] = (
        egresos_mas_relevantes_nacionales["DIAG1"]
        .replace(diagnosticos_antiguos, diagnostico_nuevo)
        .copy()
    )

Cambiando ['C341', 'C342', 'C343', 'C780', 'C782', 'D381'] a C34N
Cambiando ['Q201', 'Q202', 'Q203', 'Q204', 'Q205', 'Q206', 'Q208', 'Q209', 'Q210', 'Q211', 'Q212', 'Q213', 'Q214', 'Q218', 'Q220', 'Q221', 'Q222', 'Q223', 'Q224', 'Q225', 'Q228', 'Q230', 'Q231', 'Q233', 'Q240', 'Q241', 'Q244', 'Q245', 'Q246', 'Q248', 'Q249', 'Q250', 'Q251', 'Q253', 'Q254', 'Q255', 'Q256', 'Q257', 'Q258', 'Q259', 'Q264', 'Q268', 'Q272', 'Q273', 'Q288', 'Q289', 'Q311', 'Q320', 'Q321', 'Q330', 'Q331', 'Q332', 'Q334', 'Q338', 'Q341', 'Q348', 'Q676', 'Q677', 'Q678', 'Q765', 'Q766', 'Q767', 'Q768', 'Q769', 'Q780', 'Q790', 'Q798', 'Q839', 'Q850', 'Q858', 'Q859', 'Q874', 'Q893'] a QXXX


Previo a calcular las métricas para cada diagnóstico, es necesario agrupar algunos de ellos. Esto se debe a que ciertas incidencias fueron imposible de encontrar, como por ejemplo con:

- **I052 - Estenosis mitral con insuficiencia**: El 100% de los egresos y cantidad de pacientes será asignado al diagnóstico I051 - Insuficiencia mitral reumática. Sólamente se calculará la cantidad de casos teóricos para la insuficiencia mitral reumática.

- **I080 - Trastornos de las válvulas mitral y aórtica**: El 100% de los egresos y cantidad de pacientes será asignado al diagnóstico I340 - Insuficiencia (de la válvula) mitral y I350 - Estenosis (de la válvula) aórtica.

- **I081 - Trastornos de las válvulas mitral y tricúspide**: El 100% de los egresos y cantidad de pacientes será asignado al diagnóstico I340 - Insuficiencia (de la válvula) mitral y I361 - Insuficiencia no reumática (de la válvula) tricúspide.

In [10]:
# Reasigna I052
egresos_mas_relevantes_nacionales["DIAG1"] = (
    egresos_mas_relevantes_nacionales["DIAG1"].replace({"I052": "I051"}).copy()
)

# Reasigna I080 a I340 y I350 (Estenosis Mitral y Aortica, respectivamente)
egresos_mas_relevantes_nacionales = build_features.assign_diagnosis(
    egresos_mas_relevantes_nacionales, "I080", "I340", "I350"
)

# Reasigna I081 a I340 y I361 (Estenosis Mitral, y Valvulopatia Tricuspide)
egresos_mas_relevantes_nacionales = build_features.assign_diagnosis(
    egresos_mas_relevantes_nacionales, "I081", "I340", "I361"
)

In [11]:
# Filtra solamente los diagnosticos relevantes para el torax
egresos_mas_relevantes_torax = egresos_mas_relevantes_nacionales.query(
    "ESTABLECIMIENTO_SALUD == 112103"
).copy()

In [12]:
# Obtiene el resumen de metricas para el estrato nacional
metricas_diags_relevantes_nacionales = build_features.calculate_discharges_metrics(
    egresos_mas_relevantes_nacionales
)

# Obtiene resumen de metricas para el Torax
metricas_diags_relevantes_torax = build_features.calculate_discharges_metrics(
    egresos_mas_relevantes_torax
)

## Estimación de casos Hospitalizados utilizando % de Hospitalización del INT

En este caso, se utilizará el % de hospitalización del INT de los pacientes ambulatorios.

In [13]:
PORCENTAJE_HOSPITALIZADOS = pd.read_excel(
    "../data/external/incidencias_y_prevalencias_INT.xlsx",
    usecols=["Diagnostico", "Porcentaje Hospitalizados de Ambulatorios"],
)

PORCENTAJE_HOSPITALIZADOS = PORCENTAJE_HOSPITALIZADOS.drop_duplicates("Diagnostico")

PORCENTAJE_HOSPITALIZADOS["Diagnostico"] = (
    PORCENTAJE_HOSPITALIZADOS["Diagnostico"].str.split(" - ").str[0]
)
PORCENTAJE_HOSPITALIZADOS = PORCENTAJE_HOSPITALIZADOS.set_index("Diagnostico")[
    "Porcentaje Hospitalizados de Ambulatorios"
]

In [14]:
casos_INT_hospitalizados_porcentaje_amb = casos_area_de_influencia[
    COLUMNAS_POBLACION_INTERES.values()
].mul(PORCENTAJE_HOSPITALIZADOS, axis=0)

# Se obtiene la cantidad de egresos que debiese tener el INT, asumiendo que trabajara con la misma
# eficiencia
egresos_estimados_INT_porcentaje_amb = casos_INT_hospitalizados_porcentaje_amb.mul(
    metricas_diags_relevantes_torax["egresos_por_paciente_agrupado"]["2017-2019"], axis=0
)

# Se obtiene la cantidad de dias de estada que debiese el INT, asumiendo que trabajara con la misma
# eficiencia
dias_estada_estimados_int_porcentaje_amb = egresos_estimados_INT_porcentaje_amb.mul(
    metricas_diags_relevantes_torax["dias_estada_promedio_agrupado"]["2017-2019"], axis=0
)

# Estima la cantidad de camas necesaarias por diagnostico
camas_estimadas_int_porcentaje_amb = (
    dias_estada_estimados_int_porcentaje_amb / 365.25
) * INDICE_OCUPACION_CAMAS

# Calcula las camas totales necesarias
camas_totales_int_porcentaje_amb = camas_estimadas_int_porcentaje_amb.sum()

In [15]:
print(camas_totales_int_porcentaje_amb)

2017    251.322644
2018    255.840360
2019    260.974027
2020    266.146273
2021    269.273089
2022    271.289849
2023    273.009605
2024    274.620873
2025    276.159245
2026    277.630885
2027    279.035711
2028    280.372704
2029    281.639365
2030    282.833152
2031    284.034785
2032    285.133739
2033    286.165213
2034    287.131548
2035    288.033006
dtype: float64


## Obtener resumen para MINSAL

In [16]:
resumen_metricas_hosp = metricas_diags_relevantes_torax[
    ["egresos_por_paciente_agrupado", "dias_estada_promedio_agrupado"]
]
resumen_metricas_hosp.columns = [
    "egresos_por_paciente_agrupado_2017_a_2019",
    "dias_estada_promedio_agrupado_2017_a_2019",
]
resumen_casos = casos_INT_hospitalizados_porcentaje_amb[2035]
resumen_casos.name = "casos_hospitalizados_2035"

resumen_egresos = egresos_estimados_INT_porcentaje_amb[2035]
resumen_egresos.name = "egresos_2035"

resumen_dias_estada = dias_estada_estimados_int_porcentaje_amb[2035]
resumen_dias_estada.name = "dias_estada_totales_2035"

resumen_camas = camas_estimadas_int_porcentaje_amb[2035]
resumen_camas.name = "camas_totales_2035"

resumen_total_hosp = (
    resumen_metricas_hosp.merge(
        resumen_casos,
        how="left",
        left_index=True,
        right_index=True,
    )
    .merge(
        resumen_egresos,
        how="left",
        left_index=True,
        right_index=True,
    )
    .merge(
        resumen_dias_estada,
        how="left",
        left_index=True,
        right_index=True,
    )
    .merge(
        resumen_camas,
        how="left",
        left_index=True,
        right_index=True,
    )
)

ORDEN_COLUMNAS_RESUMEN = [
    "casos_hospitalizados_2035",
    "egresos_por_paciente_agrupado_2017_a_2019",
    "egresos_2035",
    "dias_estada_promedio_agrupado_2017_a_2019",
    "dias_estada_totales_2035",
    "camas_totales_2035",
]

resumen_total_hosp = resumen_total_hosp[ORDEN_COLUMNAS_RESUMEN]

Finalmente, une el resumen de casos teoricos desde el cuadernillo 1.0 con el resumen de este cuadernillo. Esto obtiene la tabla final para MINSAL.

In [17]:
resumen_casos_teoricos = pd.read_excel(
    "../data/interim/casos_teoricos_diagnosticos.xlsx", sheet_name="resumen_total_INT"
)

resumen_casos_teoricos["Diagnostico_codigo"] = (
    resumen_casos_teoricos["Diagnostico"].str.split(" - ").str[0]
)
resumen_casos_teoricos = resumen_casos_teoricos.set_index("Diagnostico_codigo")

resumen_minsal = resumen_casos_teoricos.merge(
    resumen_total_hosp, how="right", left_index=True, right_index=True
)

resumen_minsal = resumen_minsal.reset_index(drop=True).set_index("Diagnostico")

## Guardar archivos

In [18]:
archivos_a_guardar = {
    "metricas_relevantes_INT": metricas_diags_relevantes_torax,
    "casos_hospitalizados_INT": casos_INT_hospitalizados_porcentaje_amb,
    "egresos_estimados_INT": egresos_estimados_INT_porcentaje_amb,
    "dias_estada_estimados_INT": dias_estada_estimados_int_porcentaje_amb,
    "camas_estimadas_desglosadas_INT": camas_estimadas_int_porcentaje_amb,
    "camas_totales_INT": camas_totales_int_porcentaje_amb,
    "resumen_total_hosp_INT": resumen_total_hosp,
    "resumen_MINSAL": resumen_minsal,
}

with pd.ExcelWriter("../data/interim/estimacion_atencion_cerrada_INT.xlsx") as file:
    for nombre_hoja, df_a_guardar in archivos_a_guardar.items():
        print(f"Guardando {nombre_hoja}")
        df_a_guardar.to_excel(file, sheet_name=nombre_hoja)

Guardando metricas_relevantes_INT
Guardando casos_hospitalizados_INT
Guardando egresos_estimados_INT
Guardando dias_estada_estimados_INT
Guardando camas_estimadas_desglosadas_INT
Guardando camas_totales_INT
Guardando resumen_total_hosp_INT
Guardando resumen_MINSAL


In [19]:
DIAGS_CONGENITAS = (
    egresos_torax[
        egresos_torax["Sección"].isin(
            [
                "Q20-Q28  MALFORMACIONES CONGÉNITAS DEL APARATO CIRCULATORIO",
                "Q30-Q34  MALFORMACIONES CONGÉNITAS DEL APARATO RESPIRATORIO",
                "Q80-Q89  OTRAS MALFORMACIONES CONGÉNITAS",
            ]
        )
    ]["DIAG1"]
    .value_counts()
    .index.sort_values()
    .to_list()
)

In [21]:
import polars as pl

In [25]:
df_torax = pl.read_csv("../data/raw/6_egresos_torax/egresos_procesados_112103.csv")

In [100]:
def obtener_metricas_egresos(df, agrupacion):
    resumen = (
        df.group_by(agrupacion).agg(
            [
                pl.col("DIAG1").count().alias("n_egresos"),
                pl.col("ID_PACIENTE").n_unique().alias("n_pacientes_distintos"),
                pl.col("DIAS_ESTADA").sum().alias("dias_estada_totales"),
            ]
        )
    ).with_columns(
        [
            (pl.col("n_egresos") / pl.col("n_pacientes_distintos")).alias("egresos_por_paciente"),
            (pl.col("dias_estada_totales") / pl.col("n_egresos")).alias("dias_estada_promedio"),
        ]
    )

    return resumen


def calcular_resumen_metricas_desagregadas_y_agrupadas_en_anios(df, ano_inicio, ano_termino):
    df_filtrada = df.filter(
        (pl.col("ANO_EGRESO") >= ano_inicio) & (pl.col("ANO_EGRESO") <= ano_termino)
    )
    
    # Obtiene las metricas desagregadas por anio
    metricas_desagregadas = obtener_metricas_egresos(df_filtrada, ["ANO_EGRESO", "DIAG1"])
    metricas_desagregadas = metricas_desagregadas.sort(["ANO_EGRESO", "n_egresos"], descending=True)

    # Obtiene las metricas agrupadas en el periodo determinado
    metricas_agrupadas = obtener_metricas_egresos(df_filtrada, ["DIAG1"])
    metricas_agrupadas = metricas_agrupadas.sort(["n_egresos"], descending=True)
    
    return metricas_desagregadas, metricas_agrupadas

In [101]:
resumenes = calcular_resumen_metricas_desagregadas_y_agrupadas_en_anios(df_torax, 2017, 2019)

In [102]:
resumenes[0]

ANO_EGRESO,DIAG1,n_egresos,n_pacientes_distintos,dias_estada_totales,egresos_por_paciente,dias_estada_promedio
i64,str,u32,u32,i64,f64,f64
2019,"""Z511""",611,148,706,4.128378,1.155483
2019,"""Z514""",476,439,499,1.084282,1.048319
2019,"""I251""",323,294,2862,1.098639,8.860681
2019,"""Z291""",302,32,306,9.4375,1.013245
2019,"""C341""",220,176,2767,1.25,12.577273
…,…,…,…,…,…,…
2017,"""D848""",1,1,1,1.0,1.0
2017,"""Q248""",1,1,3,1.0,3.0
2017,"""C493""",1,1,7,1.0,7.0


In [103]:
resumenes[1]

DIAG1,n_egresos,n_pacientes_distintos,dias_estada_totales,egresos_por_paciente,dias_estada_promedio
str,u32,u32,i64,f64,f64
"""Z511""",1764,380,2046,4.642105,1.159864
"""Z514""",1259,1079,1315,1.166821,1.04448
"""I251""",978,868,8587,1.126728,8.780164
"""Z291""",646,34,650,19.0,1.006192
"""C341""",620,466,7506,1.330472,12.106452
…,…,…,…,…,…
"""J438""",1,1,9,1.0,9.0
"""Q202""",1,1,1,1.0,1.0
"""I652""",1,1,5,1.0,5.0
